In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
import sys
import importlib

# Import pysqlite3
import pysqlite3  

# Replace sqlite3 with pysqlite3
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

# Verify that sqlite3 now points to pysqlite3
import sqlite3
print(sqlite3.__file__)

from langchain_chroma import Chroma  # Importing it after as I am having issues with sqlite

/usr/local/python/3.12.1/lib/python3.12/site-packages/pysqlite3/__init__.py


In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

### I will be creating a text file in which all the tagged description are separated by a new line. Bascially we want to create a embedding in which every description is a new value.

In [6]:
books["tagged_description"].to_csv("tagged_description.txt", sep="\n", index=False, header=False)

In [7]:
raw_documents = TextLoader("tagged_description.txt", encoding = 'UTF-8').load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

Created a chunk of size 199, which is longer than the specified 0
Created a chunk of size 767, which is longer than the specified 0
Created a chunk of size 553, which is longer than the specified 0
Created a chunk of size 227, which is longer than the specified 0
Created a chunk of size 233, which is longer than the specified 0
Created a chunk of size 590, which is longer than the specified 0
Created a chunk of size 238, which is longer than the specified 0
Created a chunk of size 299, which is longer than the specified 0
Created a chunk of size 726, which is longer than the specified 0
Created a chunk of size 1805, which is longer than the specified 0
Created a chunk of size 1017, which is longer than the specified 0
Created a chunk of size 684, which is longer than the specified 0
Created a chunk of size 739, which is longer than the specified 0
Created a chunk of size 236, which is longer than the specified 0
Created a chunk of size 449, which is longer than the specified 0
Created 

In [8]:
documents[0]
documents_500 = documents[0:500] # For test purpose only, in principal all documents should be used

### I will be creating embedings for the database using chroma

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db_books = Chroma.from_documents(
    documents_500,
    embedding=embedding_model
)

In [10]:
db_books

In [11]:
query = "A book to teach childern about nature"
docs = db_books.similarity_search(query, k=10)
docs

[Document(id='5475f701-f845-42b0-a41b-44c081c82173', metadata={'source': 'tagged_description.txt'}, page_content='"9780067575208 First published more than three decades ago, this reissue of Rachel Carson\'s award-winning classic brings her unique vision to a new generation of readers. Stunning new photographs by Nick Kelsh beautifully complement Carson\'s intimate account of adventures with her young nephew, Roger, as they enjoy walks along the rocky coast of Maine and through dense forests and open fields, observing wildlife, strange plants, moonlight and storm clouds, and listening to the ""living music"" of insects in the underbrush. ""If a child is to keep alive his inborn sense of wonder."" Writes Carson, ""he needs the companionship of at least one adult who can share it, rediscovering with him the joy, excitement and mystery of the world we live in."" The Sense of Wonder is a refreshing antidote to indifference and a guide to capturing the simple power of discovery that Carson v

In [12]:
# Function for retreiving books based on the query provided

def retrieve_semantic_recommendations(
    query: str,
    top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])] # striping the isbn from the page content and storing it in books_list

    return books[books["isbn13"].isin(books_list)].head(top_k)

In [13]:
retrieve_semantic_recommendations("A book to teach childern about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tagged_description
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...
31,9780007105045,0007105045,Tree and Leaf,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,Tree and Leaf: The Homecoming of Beorhtnoth : ...,"9780007105045 ""The two works 'On fairy-stories..."
57,9780007149124,0007149123,Tales from the Perilous Realm,John Ronald Reuel Tolkien,"Fairy tales, English",http://books.google.com/books/content?id=Wla7N...,"Never before published in a single volume, Tol...",2002.0,4.08,178.0,2964.0,Tales from the Perilous Realm,9780007149124 Never before published in a sing...
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240 It should be the most natural th...
86,9780060000141,0060000147,Poppy's Return,Avi,Juvenile Fiction,http://books.google.com/books/content?id=XbcMJ...,"There's trouble at Gray House, the girlhood ho...",2006.0,3.99,256.0,1086.0,Poppy's Return,"9780060000141 There's trouble at Gray House, t..."
99,9780060094935,0060094931,The Bromeliad Trilogy,Terry Pratchett,Juvenile Fiction,http://books.google.com/books/content?id=IsFpv...,In a world whose seasons are defined by Christ...,2003.0,4.06,502.0,5681.0,"The Bromeliad Trilogy: Truckers, Diggers, and ...",9780060094935 In a world whose seasons are def...
108,9780060278427,0060278420,Laura's Album,William Anderson,Juvenile Nonfiction,http://books.google.com/books/content?id=_zTkq...,Though best known as the author of the Little ...,1998.0,4.30,80.0,713.0,Laura's Album: A Remembrance Scrapbook of Laur...,9780060278427 Though best known as the author ...
113,9780060507404,0060507403,The Cheese Monkeys,Chip Kidd,Fiction,http://books.google.com/books/content?id=wkCm2...,"After 15 years of designing more than 1,500 bo...",2002.0,3.75,288.0,4570.0,The Cheese Monkeys: A Novel in Two Semesters,9780060507404 After 15 years of designing more...
143,9780060546571,0060546573,Three Rotten Eggs,Gregory Maguire,Juvenile Fiction,http://books.google.com/books/content?id=t2pWl...,The students of Miss Earth's class in rural Ve...,2005.0,3.74,240.0,76.0,Three Rotten Eggs,9780060546571 The students of Miss Earth's cla...
146,9780060556501,0060556501,"The Lion, the Witch and the Wardrobe (picture ...",C. S. Lewis,Juvenile Fiction,http://books.google.com/books/content?id=FlSpp...,Narnia: A magical land full of wonder and exci...,2004.0,4.19,48.0,5012.0,"The Lion, the Witch and the Wardrobe (picture ...",9780060556501 Narnia: A magical land full of w...


In [14]:
category_mapping = {
    'Fiction': "Fiction",
    'Juvenile Fiction': "Childern's Fiction",
    'Biography & Autobiography': "Nonfiction",
    'History': "Nonfiction",
    'Literary Criticism': "Nonfiction",
    'Philosophy': "Nonfiction",
    'Religion': "Nonfiction",
    'Comics & Graphic Novels': "Fiction",
    'Drama': "Fiction",
    'Juvenile Nonfiction': "Children's Nonfiction",
    'Science': "Nonfiction",
    'Poetry': "Fiction"
}

books["simple_categories"] = books["categories"].map(category_mapping)

In [15]:
# Use a pipeline as a high-level helper
from transformers import pipeline

fiction_categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cpu


In [16]:
sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[0]

In [ ]:
pipe(sequence, fiction_categories) # We are sending sequence which is a description of the first entry whose simple_categories is Fiction
                                   # The fiction_categories are the labels on which zero_shot classification would happen

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [18]:
def generate_predictions(sequence, categories):
    predictions = pipe(sequence, categories)       # So basically we are getting an object with description, labels and scores attributes. {"description": ...., "labels":['Fiction', 'Nonfiction'], "scores": [0.8745345, 0.1536345] }
    max_index = np.argmax(predictions["scores"])   # We are taking the index for the highest score , it will be either 0 or 1. Based on the position of the higest score
    max_label = predictions["labels"][max_index]   # Then we are taking the label out based on the highest score. 0 = Fiction, 1 = Nonfiction
    return max_label

### Checking Classifer Accuracy

In [19]:
from tqdm import tqdm  # To see how long would it take
import numpy as np

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    actual_cats += ["Fiction"]

100%|██████████| 300/300 [12:52<00:00,  2.58s/it]


In [20]:
for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Nonfiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    actual_cats += ["Nonfiction"]

100%|██████████| 300/300 [14:24<00:00,  2.88s/it]


In [21]:
predictions_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})

In [22]:
# Putting 1 where prediction was correct
predictions_df["corrected_predictions"] = (
    np.where(predictions_df["actual_categories"] == predictions_df["predicted_categories"], 1, 0)
)

# Taking sum of all predictions and then dividing it by the length of the whole dataframe
predictions_df["corrected_predictions"].sum() / len(predictions_df)

0.7783333333333333

### Filling entries with missing categories

In [31]:
isbns = []
predicted_cats = []

# Taking a dataframe with isbn13 and description of the rows with missing categories
missing_cats = books.loc[books["simple_categories"].isna(), ["isbn13", "description"]].reset_index(drop=True)

In [33]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["description"][i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    isbns += [missing_cats["isbn13"][i]]

In [34]:
missing_predicted_df = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_cats})

In [35]:
books = pd.merge(books, missing_predicted_df, on="isbn13", how="left")

# Adding values from the predicted_categories column if the simple_categories is missing
books["simple_categories"] = np.where(
                                        books["simple_categories"].isna(), 
                                        books["predicted_categories"], 
                                        books["simple_categories"]
                                    )
books = books.drop(columns = ["predicted_categories"])

In [36]:
books.to_csv("books_with_categories.csv", index=False)